<a href="https://colab.research.google.com/github/dustiny5/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module2-sampling-confidence-intervals-and-hypothesis-testing/LS_DS_142_Sampling_Confidence_Intervals_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 142
## Sampling, Confidence Intervals, and Hypothesis Testing

## Prepare - examine other available hypothesis tests

If you had to pick a single hypothesis test in your toolbox, t-test would probably be the best choice - but the good news is you don't have to pick just one! Here's some of the others to be aware of:

In [2]:
import numpy as np
from scipy.stats import chisquare  # One-way chi square test

# Chi square can take any crosstab/table and test the independence of rows/cols
# The null hypothesis is that the rows/cols are independent -> low chi square
# The alternative is that there is a dependence -> high chi square
# Be aware! Chi square does *not* tell you direction/causation

ind_obs = np.array([[1, 1], [2, 2]]).T
print(ind_obs)
print(chisquare(ind_obs, axis=None))

dep_obs = np.array([[16, 18, 16, 14, 12, 12], [32, 24, 16, 28, 20, 24]]).T
print(dep_obs)
print(chisquare(dep_obs, axis=None))

[[1 2]
 [1 2]]
Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)
[[16 32]
 [18 24]
 [16 16]
 [14 28]
 [12 20]
 [12 24]]
Power_divergenceResult(statistic=23.31034482758621, pvalue=0.015975692534127565)


In [3]:
# Distribution tests:
# We often assume that something is normal, but it can be important to *check*

# For example, later on with predictive modeling, a typical assumption is that
# residuals (prediction errors) are normal - checking is a good diagnostic

from scipy.stats import normaltest
# Poisson models arrival times and is related to the binomial (coinflip)
sample = np.random.poisson(5, 1000)
print(normaltest(sample))  # Pretty clearly not normal

NormaltestResult(statistic=48.9015083303999, pvalue=2.4053173707340052e-11)


In [4]:
# Kruskal-Wallis H-test - compare the median rank between 2+ groups
# Can be applied to ranking decisions/outcomes/recommendations
# The underlying math comes from chi-square distribution, and is best for n>5
from scipy.stats import kruskal

x1 = [1, 3, 5, 7, 9]
y1 = [2, 4, 6, 8, 10]
print(kruskal(x1, y1))  # x1 is a little better, but not "significantly" so

x2 = [1, 1, 1]
y2 = [2, 2, 2]
z = [2, 2]  # Hey, a third group, and of different size!
print(kruskal(x2, y2, z))  # x clearly dominates

KruskalResult(statistic=0.2727272727272734, pvalue=0.6015081344405895)
KruskalResult(statistic=7.0, pvalue=0.0301973834223185)


And there's many more! `scipy.stats` is fairly comprehensive, though there are even more available if you delve into the extended world of statistics packages. As tests get increasingly obscure and specialized, the importance of knowing them by heart becomes small - but being able to look them up and figure them out when they *are* relevant is still important.

## Live Lecture - let's explore some more of scipy.stats

Candidate topics to explore:

- `scipy.stats.chi2` - the Chi-squared distribution, which we can use to reproduce the Chi-squared test
- Calculate the Chi-Squared test statistic "by hand" (with code), and feed it into `chi2`
- Build a confidence interval with `stats.t.ppf`, the t-distribution percentile point function (the inverse of the CDF) - we can write a function to return a tuple of `(mean, lower bound, upper bound)` that you can then use for the assignment (visualizing confidence intervals)

In [7]:
# Taking requests! Come to lecture with a topic or problem and we'll try it.
gender = ['male', 'male', 'male', 'female', 'female', 'female']
eats_outside = ['outside', 'inside', 'inside', 'inside', 'outside', 'outside']

import pandas as pd

df= pd.DataFrame({'gender':gender, 'preference':eats_outside})
df.head(6)

,gender,preference
0,male,outside
1,male,inside
2,male,inside
3,female,inside
4,female,outside
5,female,outside


In [10]:
table = pd.crosstab(df['gender'],df['preference'])
table

preference,inside,outside
gender,,
female,1,2
male,2,1


In [9]:
pd.crosstab(df['gender'],df['preference'], margins=True) #margins sum up rows and cols

preference,inside,outside,All
gender,,,
female,1,2,3
male,2,1,3
All,3,3,6


In [14]:
df = df.replace({'male':0,'female':1,'outside':0,'inside':1})
df

,gender,preference
0,0,0
1,0,1
2,0,1
3,1,1
4,1,0
5,1,0


In [0]:
#chi2 = sum ((observed - expected)**2 / expected)

In [27]:
import numpy as np
pd.crosstab(df['gender'],df['preference'], margins=True)

#Total number of people / total number of cells(6/4)
expected = np.array([[1.5, 1.5],
            [1.5, 1.5]])

#Marginal Proportion of the first row
# obs / total = 3 males / 6 humans
pd.crosstab(df['gender'],df['preference'], margins=True, normalize='all')

#Observed is a table of the marginal proportions of each subgroup
observed = np.array([[0.5, 0.5],
            [0.5, 0.5]])

deviation = observed - expected
print(deviation)

deviation_squared = deviation**2
print('deviation squared: \n', deviation_squared)

fraction = (deviation_squared / expected)
print('fraction: \n', fraction)

chi2 = fraction.sum()
print(chi2/4)

[[-1. -1.]
 [-1. -1.]]
deviation squared: 
 [[1. 1.]
 [1. 1.]]
fraction: 
 [[0.66666667 0.66666667]
 [0.66666667 0.66666667]]
0.6666666666666666


In [28]:
chi_data = [[1,2],[2,1]]

from scipy.stats import chisquare

chisquare(chi_data, axis=None)

Power_divergenceResult(statistic=0.6666666666666666, pvalue=0.8810148425137847)

In [29]:
from scipy.stats import chi2_contingency

chi2statistic, pvalue, dof, observed = chi2_contingency(table)

print('chi2 stat', chi2statistic)
print('pvalue', pvalue)
print('degrees of freedom', dof)
print('contingency table', observed)

chi2 stat 0.0
pvalue 1.0
degrees of freedom 1
contingency table [[1.5 1.5]
 [1.5 1.5]]


#Confidence Interval

In [32]:
coinflips = np.random.binomial(n=1, p=.5, size=100)
print(coinflips)

[1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1
 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 1
 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 0 1]


In [34]:
import scipy.stats as stats
stats.ttest_1samp(coinflips, 0.5)
#H0 = Fair coin with 50% H1 = Not a fair coin

Ttest_1sampResult(statistic=-0.39831375340784614, pvalue=0.6912566363051549)

## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):

1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)

In [0]:
import scipy
import numpy as np

In [37]:
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
columns = {0:'class_name',1:'handicapped_infants',2:'water_project_cost_sharing',3:'adoption_of_the_budget_resolution',
           4:'physician_free_freeze',5:'el_salvador_aid',6:'religious_groups_in_schools',7:'anti_satellite_test_ban',
           8:'aid_to_nicaraguan_contras',9:'mx_missile',10:'immigration',11:'synfuels_corp_cutback',12:'edu_spending',
           13:'superfund_right_to_sue',14:'crime',15:'duty_free_exports',16:'export_admin._act_s.africa'}

#Clean data
df = pd.read_csv(url, header=None, na_values='?')
print(df.shape)
df.head()

(435, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
#Rename columns and change class_name to binary
df = df.rename(columns=columns)
df['class_name'] = (df['class_name']=='republican').astype(int)

In [0]:
#Rename classname to is_republican
df = df.rename(columns={'class_name':'is_republican'})

#Replace n to 0 and y to 1
df = df.replace(['n','y'],[0,1])

In [44]:
df.head()

,is_republican,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_free_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cutback,edu_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin._act_s.africa
0,1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,0,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,0,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
dem = df[df['is_republican'] == 0]
rep =  df[df['is_republican'] == 1]

In [61]:
scipy.stats.ttest_1samp(dem['handicapped_infants'], df['handicapped_infants'].mean(), nan_policy='omit')

Ttest_1sampResult(statistic=5.330480809203451, pvalue=2.144451085424599e-07)

In [0]:
import numpy as np
from scipy import stats

def confidence_interval(data, confidence=0.95):
  """
  Calculate a confidence interval around a sample mean for given data.
  Using t-distribution and two-tailed test, default 95% confidence. 
  
  Arguments:
    data - iterable (list or numpy array) of sample observations
    confidence - level of confidence for the interval
  
  Returns:
    tuple of (mean, lower bound, upper bound)
  """
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  stderr = stats.sem(data)
  interval = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
  return (mean, mean - interval, mean + interval)

def report_confidence_interval(confidence_interval):
  """
  Return a string with a pretty report of a confidence interval.
  
  Arguments:
    confidence_interval - tuple of (mean, lower bound, upper bound)
  
  Returns:
    None, but prints to screen the report
  """
  #print('Mean: {}'.format(confidence_interval[0]))
  #print('Lower bound: {}'.format(confidence_interval[1]))
  #print('Upper bound: {}'.format(confidence_interval[2]))
  s = "our mean lies in the interval [{:.2}, {:.2}]".format(
      confidence_interval[1], confidence_interval[2])
  return s

In [69]:
conf_hand_inf = confidence_interval(dem['handicapped_infants'].dropna())
conf_hand_inf

#I am 95% confident that my mean, 0.6, will be between the 0.54 and 0.66

(0.6046511627906976, 0.5445926997425515, 0.6647096258388437)

In [70]:
report_confidence_interval(conf_hand_inf)

'our mean lies in the interval [0.54, 0.66]'

In [78]:
import matplotlib.pyplot as plt

mean, std = dem['handicapped_infants'].mean(), dem['handicapped_infants'].std()

n = dem['handicapped_infants'].shape[1]
yerr = std / np.sqrt(n) * stats.t.ppf(1-0.05/2, n - 1)
plt.figure()
plt.bar(range(dem['handicapped_infants'].shape[0]), mean, yerr = yerr)

IndexError: ignored